# get validation, n downsample indices

In [2]:
import json
import os
import gc
import numpy as np
import pandas as pd

def downsampling(positive_idx, negative_idx, ratio):
    idx = np.random.choice(negative_idx, int(ratio*len(negative_idx)))
    idx = np.concatenate((idx, positive_idx))
    return np.sort(idx).astype(int).tolist()


PATH1 = '/home/kai/data/kaggle/talkingdata/data/'
PATH0 = '/home/kai/data/kaggle/talkingdata/haoyandata/'
PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'

file = {}
train = pd.read_feather(PATH0 + 'train_cleaned.ftr')
# train = pd.read_csv(PATH1 + 'train_cleaned.csv', nrows=10000)
val_idx = (train['day'] == 9) & (train['hour'] == 13) | (train['hour'] == 17) | (train['hour'] == 21)
file['val_idx'] = val_idx[val_idx].index.values.astype(int).tolist()


# Downsampling
train = train[~val_idx]
negative = list(train[train['is_attributed']==0].index.values)
positive = list(train[train['is_attributed']==1].index.values)
ratio = len(positive)/len(negative)
idx_list = []
for i in range(5):
    idx_list.append(downsampling(positive, negative, ratio))
    print(i)
file['downsampling'] = idx_list
with open(PATH + 'idx_file.json', 'w') as f:
    json.dump(file, f)
print(ratio)
print('done')

0
1
2
3
4
0.00246100785192494
done


# split files into several by validation and down sample indices

In [2]:
import os
import gc
import json
import pandas as pd
from tqdm import tqdm

ORIGINAL_FILE_PATH = '/home/kai/data/kaggle/talkingdata/haoyandata/'
DOWNSAMPLING_PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'
suffix = 'ftr'
with open(DOWNSAMPLING_PATH + 'idx_file.json', 'r') as f:
    data = json.load(f)
    val_idx = data['val_idx']
    downsampling_idx = data['downsampling']

print('making directory')
directory = os.path.dirname((DOWNSAMPLING_PATH + 'val/'))
if not os.path.exists(directory):
    os.makedirs(directory)

for i in range(len(downsampling_idx)):
    directory = os.path.dirname((DOWNSAMPLING_PATH + str(i) + '/'))
    if not os.path.exists(directory):
        os.makedirs(directory)
    
print('down sampling')
l = 0
for file in os.listdir(ORIGINAL_FILE_PATH):
    l += 1
    print(l, file)
    if file.split('.')[-1] == suffix and file.split('.')[0].split('__')[0] == 'train':
        data = pd.read_feather(ORIGINAL_FILE_PATH + file)
        # validation
        data.loc[val_idx].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + 'val/' + file)
        # down sampling
        for i, x in enumerate(downsampling_idx):
            data.loc[x].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + str(i) + '/' + file)
        del data
        print(file)
        gc.collect()

making directory
down sampling
1 test_supplement__woe_ip_device_channel.ftr
2 train__woe_ip_app_channel.ftr
train__woe_ip_app_channel.ftr
3 test_supplement__range_count_ip_device_os_dayhourminute10.ftr
4 test_supplement__woe_device.ftr
5 train__woe_ip_device_nextClickLeakDayFlt.ftr
train__woe_ip_device_nextClickLeakDayFlt.ftr
6 test_supplement__woe_ip_app_os_channel.ftr
7 train__nunique_machine_app.ftr
train__nunique_machine_app.ftr
8 test_supplement__woe_app_os_channel.ftr
9 test_supplement__unique_ratio_day_ip_channel.ftr
10 test_supplement__woe_os_channel_nextClickLeakDayFlt.ftr
11 train__woe_os_channel_nextClickLeakDayFlt.ftr
train__woe_os_channel_nextClickLeakDayFlt.ftr
12 test_supplement__count_ip.ftr
13 test_supplement__count_app_device_channel_day_hour.ftr
14 train__variance_ip_device_hour.ftr
train__variance_ip_device_hour.ftr
15 train__range_count_ip_device_os_dayhourminute10.ftr
train__range_count_ip_device_os_dayhourminute10.ftr
16 test_supplement__variance_ip_device_hour.f

train__woe_ip_app_device_channel.ftr
133 test_supplement__woe_ip_os_channel.ftr
134 train__unique_ratio_day_ip_os.ftr
train__unique_ratio_day_ip_os.ftr
135 test_supplement__unique_ratio_machine_ip.ftr
136 train__range_count_ip_dayhourminute10.ftr
train__range_count_ip_dayhourminute10.ftr
137 test_supplement__woe_ip_app_nextClickLeakDayFlt.ftr
138 train__woe_ip_nextClickLeakDayFlt.ftr
train__woe_ip_nextClickLeakDayFlt.ftr
139 test_supplement__woe_ip.ftr
140 test_supplement__count_ratio_ip_machine.ftr
141 test_supplement__nunique_day_ip_machine.ftr
142 test_supplement__range_count_ip_day.ftr
143 train__woe_device.ftr
train__woe_device.ftr
144 train__woe_app_device_nextClickLeakDayFlt.ftr
train__woe_app_device_nextClickLeakDayFlt.ftr
145 train_cleaned.ftr
146 train__unique_ratio_day_ip_device.ftr
train__unique_ratio_day_ip_device.ftr
147 train__count_ip_app_os_channel_day.ftr
train__count_ip_app_os_channel_day.ftr
148 train__count_ip_device_os_day_hourminute10.ftr
train__count_ip_device_o

In [3]:
from tqdm import tqdm
print('down sampling')
for file in tqdm(os.listdir(ORIGINAL_FILE_PATH)):
    if file.split('.')[-1] == suffix and file.split('.')[0] == 'train_cleaned':
        print(file)
        data = pd.read_feather(ORIGINAL_FILE_PATH + file)
        # validation
        data.loc[val_idx].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + 'val/' + file)
        # down sampling
        for i, x in enumerate(downsampling_idx):
            data.loc[x].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + str(i) + '/' + file)
        del data
        print('done')
        gc.collect()

  0%|          | 0/198 [00:00<?, ?it/s]

down sampling
train_cleaned.ftr


100%|██████████| 198/198 [01:21<00:00,  2.42it/s]

done


# concatenate val, down sampling files into one

In [4]:
import os
import gc
import pandas as pd

PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'
suffix = 'ftr'
dir_list = ['val/', '1/', '2/', '3/', '4/', '0/']
for x in dir_list:
    CURRENT_PATH = PATH + x
    file_name = PATH + x[:-1] + '.' + suffix
    df = pd.read_feather(CURRENT_PATH + 'train_cleaned.ftr')
    for y in tqdm(os.listdir(CURRENT_PATH)):
        splitted_y = y.split('.')
        if splitted_y[0] != 'train_cleaned' and splitted_y[1] == suffix:
            tmp = pd.read_feather(CURRENT_PATH + y)
            df[splitted_y[0].split('__')[1]] = tmp
            del tmp
    print('converting')
    df.to_feather(file_name)
    del df
    gc.collect()
    print(file_name)

100%|██████████| 99/99 [00:30<00:00,  3.20it/s]


converting


 26%|██▋       | 26/99 [00:00<00:00, 248.17it/s]

/home/kai/data/kaggle/talkingdata/downsampling/val.ftr


100%|██████████| 99/99 [00:00<00:00, 107.59it/s]


...

converting


 25%|██▌       | 25/99 [00:00<00:00, 249.63it/s]

/home/kai/data/kaggle/talkingdata/downsampling/4.ftr


100%|██████████| 99/99 [00:00<00:00, 107.95it/s]


converting
/home/kai/data/kaggle/talkingdata/downsampling/0.ftr


# test concate

In [5]:
import pandas as pd
import os
import gc

PATH = '/home/kai/data/kaggle/talkingdata/haoyandata/'
file_name = '/home/kai/data/kaggle/talkingdata/4thplace/test_cleaned.ftr'
df = pd.read_feather(PATH + 'test_cleaned.ftr')
for x in os.listdir(PATH):
    splitted = x.split('__')
    if splitted[0] == 'test_supplement':
        print(x)
        tmp = pd.read_feather(PATH + x)
        df[splitted[1].split('.')[0]] = tmp
        del tmp
        gc.collect()
print('converting')
df.to_feather(file_name)
del df
gc.collect()
print(file_name)

test_supplement__woe_ip_device_channel.ftr
test_supplement__range_count_ip_device_os_dayhourminute10.ftr
test_supplement__woe_device.ftr
test_supplement__woe_ip_app_os_channel.ftr
test_supplement__woe_app_os_channel.ftr
test_supplement__unique_ratio_day_ip_channel.ftr
test_supplement__woe_os_channel_nextClickLeakDayFlt.ftr
test_supplement__count_ip.ftr
test_supplement__count_app_device_channel_day_hour.ftr
test_supplement__variance_ip_device_hour.ftr
test_supplement__count_ratio_ip_channel.ftr
test_supplement__woe_ip_device_os_channel.ftr
test_supplement__woe_ip_device_nextClickLeakDayFlt.ftr
test_supplement__nunique_day_ip_os.ftr
test_supplement__count_app_device_day_hour.ftr
test_supplement__woe_ip_app_device_channel.ftr
test_supplement__woe_ip_app.ftr
test_supplement__woe_app_device_nextClickLeakDayFlt.ftr
test_supplement__woe_app_device_channel.ftr
test_supplement__range_count_app_os_channel_dayhourminute10.ftr
test_supplement__woe_ip_app_channel.ftr
test_supplement__range_count_ip

# concat val train without down sampling

In [1]:
import os
import pandas as pd
import gc
PATH = '/home/kai/data/kaggle/talkingdata/haoyandata/'

train = pd.read_feather(PATH + 'train_cleaned.ftr')
val_idx = (train['day'] == 9) & (train['hour'] == 13) | (train['hour'] == 17) | (train['hour'] == 21)
idx = ~val_idx
val = train[val_idx]
train = train[idx]
print(gc.collect())

print('start')
for x in os.listdir(PATH):
    tmp = x.split('.')
    if tmp[-1] == 'ftr':
        ttmp = tmp[0].split('__')
        if ttmp[0] == 'train':
            f = pd.read_feather(PATH + x)
            val[ttmp[1]] = f[val_idx]
            train[ttmp[1]] = f[idx]
            del f
            gc.collect()
            print(x, ttmp[1])
val.reset_index(drop=True).to_feather('/home/kai/data/kaggle/talkingdata/4thplace/val_cleaned.ftr')
train.reset_index(drop=True).to_feather('/home/kai/data/kaggle/talkingdata/4thplace/train_cleaned.ftr')

0
start
train__woe_ip_app_channel.ftr woe_ip_app_channel
train__woe_ip_device_nextClickLeakDayFlt.ftr woe_ip_device_nextClickLeakDayFlt
train__nunique_machine_app.ftr nunique_machine_app
train__woe_os_channel_nextClickLeakDayFlt.ftr woe_os_channel_nextClickLeakDayFlt
train__variance_ip_device_hour.ftr variance_ip_device_hour
train__range_count_ip_device_os_dayhourminute10.ftr range_count_ip_device_os_dayhourminute10
train__woe_app_device_os_channel.ftr woe_app_device_os_channel
train__woe_channel.ftr woe_channel
train__count_ip_app_os_channel.ftr count_ip_app_os_channel
train__nunique_day_ip_os.ftr nunique_day_ip_os
train__count_app_channel.ftr count_app_channel
train__woe_app_nextClickLeakDayFlt.ftr woe_app_nextClickLeakDayFlt
train__woe_ip_channel.ftr woe_ip_channel
train__unique_ratio_day_ip_machine.ftr unique_ratio_day_ip_machine
train__nunique_machine_channel.ftr nunique_machine_channel
train__time_to_n_next_click_1_day_ip_app_device_os.ftr time_to_n_next_click_1_day_ip_app_device

# update

In [ ]:
import os
import pandas as pd
import gc
PATH = '/home/kai/data/kaggle/talkingdata/haoyandata/'

train = pd.read_feather('/home/kai/data/kaggle/talkingdata/4thplace/train_cleaned.ftr')
val = pd.read_feather('/home/kai/data/kaggle/talkingdata/4thplace/val_cleaned.ftr')
test = pd.read_feather('/home/kai/data/kaggle/talkingdata/4thplace/test_cleaned.ftr')
val_idx = (train['day'] == 9) & (train['hour'] == 13) | (train['hour'] == 17) | (train['hour'] == 21)
print(gc.collect())

print('start')
for x in os.listdir(PATH):
    tmp = x.split('.')
    if tmp[-1] == 'ftr':
        ttmp = tmp[0].split('__')
#         if ttmp[0] == 'train':
        if 'woe' in ttmp[1]:
            f = pd.read_feather(PATH + x)
            if ttmp[0] == 'train':
                val[ttmp[1]] = f[val_idx]
                train[ttmp[1]] = f[idx]
            elif ttmp[0] == 'test_supplement':
                test[ttmp[1]] = f
            del f
            gc.collect()
            print(x, ttmp[1])
val.reset_index(drop=True).to_feather('/home/kai/data/kaggle/talkingdata/4thplace/val_cleaned1.ftr')
train.reset_index(drop=True).to_feather('/home/kai/data/kaggle/talkingdata/4thplace/train_cleaned1.ftr')
test.reset_index(drop=True).to_feather('/home/kai/data/kaggle/talkingdata/4thplace/test_cleaned1.ftr')

# first place features

In [1]:
import os
import gc
import json
import pandas as pd
from tqdm import tqdm

ORIGINAL_FILE_PATH = '/home/kai/data/kaggle/talkingdata/haoyandata_1st/'
DOWNSAMPLING_PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'
suffix = 'ftr'
with open(DOWNSAMPLING_PATH + 'idx_file.json', 'r') as f:
    data = json.load(f)
    val_idx = data['val_idx']
    downsampling_idx = data['downsampling']

print('making directory')
directory = os.path.dirname((DOWNSAMPLING_PATH + 'val/'))
if not os.path.exists(directory):
    os.makedirs(directory)

for i in range(len(downsampling_idx)):
    directory = os.path.dirname((DOWNSAMPLING_PATH + str(i) + '/'))
    if not os.path.exists(directory):
        os.makedirs(directory)
    
print('down sampling')
l = 0
for file in os.listdir(ORIGINAL_FILE_PATH):
    l += 1
    print(l, file)
    if file.split('.')[-1] == suffix and file.split('.')[0].split('__')[0] == 'train':
        data = pd.read_feather(ORIGINAL_FILE_PATH + file)
        # validation
        data.loc[val_idx].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + 'val/' + file)
        # down sampling
        for i, x in enumerate(downsampling_idx):
            data.loc[x].reset_index(drop=True).to_feather(DOWNSAMPLING_PATH + str(i) + '/' + file)
        del data
        print(file)
        gc.collect()

making directory
down sampling
1 train__click_count_in_previous_n_hour_ip_device_app_1.ftr
train__click_count_in_previous_n_hour_ip_device_app_1.ftr
2 test_supplement__click_count_in_previous_n_hour_ip_device_app_channel_1.ftr
3 train__click_count_in_previous_n_hour_device_6.ftr
train__click_count_in_previous_n_hour_device_6.ftr
4 test_supplement__click_count_in_next_n_hour_device_os_channel_6.ftr
5 test_supplement__click_count_in_next_n_hour_device_app_os_1.ftr
6 test_supplement__click_count_in_previous_n_hour_device_app_channel_1.ftr
7 train__click_count_in_previous_n_hour_ip_os_6.ftr
train__click_count_in_previous_n_hour_ip_os_6.ftr
8 train__click_count_in_next_n_hour_os_1.ftr
train__click_count_in_next_n_hour_os_1.ftr
9 train__click_count_in_previous_n_hour_ip_device_channel_1.ftr
train__click_count_in_previous_n_hour_ip_device_channel_1.ftr
10 test_supplement__click_count_in_next_n_hour_device_os_6.ftr
11 test_supplement__click_count_in_next_n_hour_ip_device_app_os_1.ftr
12 test_s

train__click_count_in_previous_n_hour_ip_app_1.ftr
95 test_supplement__click_count_in_previous_n_hour_device_channel_1.ftr
96 train__click_count_in_previous_n_hour_ip_device_os_6.ftr
train__click_count_in_previous_n_hour_ip_device_os_6.ftr
97 test_supplement__click_count_in_previous_n_hour_ip_channel_1.ftr
98 train__click_count_in_previous_n_hour_ip_device_channel_6.ftr
train__click_count_in_previous_n_hour_ip_device_channel_6.ftr
99 train__click_count_in_next_n_hour_app_channel_6.ftr
train__click_count_in_next_n_hour_app_channel_6.ftr
100 train__click_count_in_next_n_hour_ip_channel_6.ftr
train__click_count_in_next_n_hour_ip_channel_6.ftr
101 train__click_count_in_next_n_hour_ip_device_app_channel_6.ftr
train__click_count_in_next_n_hour_ip_device_app_channel_6.ftr
102 test_supplement__click_count_in_previous_n_hour_os_1.ftr
103 test_supplement__click_count_in_next_n_hour_ip_device_os_6.ftr
104 train__click_count_in_next_n_hour_channel_1.ftr
train__click_count_in_next_n_hour_channel_1.

train__click_count_in_next_n_hour_ip_device_6.ftr
188 test_supplement__click_count_in_next_n_hour_ip_channel_1.ftr
189 train__click_count_in_next_n_hour_device_app_os_6.ftr
train__click_count_in_next_n_hour_device_app_os_6.ftr
190 train__click_count_in_next_n_hour_ip_device_os_channel_6.ftr
train__click_count_in_next_n_hour_ip_device_os_channel_6.ftr
191 train__click_count_in_previous_n_hour_ip_app_6.ftr
train__click_count_in_previous_n_hour_ip_app_6.ftr
192 test_supplement__click_count_in_previous_n_hour_ip_app_1.ftr
193 train__click_count_in_next_n_hour_device_os_1.ftr
train__click_count_in_next_n_hour_device_os_1.ftr
194 test_supplement__click_count_in_next_n_hour_ip_app_channel_6.ftr
195 train__click_count_in_next_n_hour_device_app_channel_1.ftr
train__click_count_in_next_n_hour_device_app_channel_1.ftr
196 test_supplement__click_count_in_previous_n_hour_ip_device_app_channel_6.ftr
197 train__click_count_in_next_n_hour_ip_device_app_channel_1.ftr
train__click_count_in_next_n_hour_i

# val and train

In [2]:
import os
import gc
import pandas as pd

PATH = '/home/kai/data/kaggle/talkingdata/downsampling/'
suffix = 'ftr'
dir_list = ['val/', '1/', '2/', '3/', '4/', '0/']
for x in dir_list:
    CURRENT_PATH = PATH + x
    file_name = PATH + x[:-1] + '1.' + suffix
    df = pd.read_feather(CURRENT_PATH + 'train_cleaned.ftr')
    for y in tqdm(os.listdir(CURRENT_PATH)):
        splitted_y = y.split('.')
        if splitted_y[0] != 'train_cleaned' and splitted_y[1] == suffix:
            tmp = pd.read_feather(CURRENT_PATH + y)
            df[splitted_y[0].split('__')[1]] = tmp
            del tmp
    print('converting')
    df.to_feather(file_name)
    del df
    gc.collect()
    print(file_name)

100%|██████████| 213/213 [01:07<00:00,  3.18it/s]


converting
/home/kai/data/kaggle/talkingdata/downsampling/val1.ftr


100%|██████████| 213/213 [00:04<00:00, 43.22it/s]


converting
/home/kai/data/kaggle/talkingdata/downsampling/11.ftr


100%|██████████| 213/213 [00:03<00:00, 59.87it/s]


...

converting


 10%|█         | 22/213 [00:00<00:00, 219.05it/s]

/home/kai/data/kaggle/talkingdata/downsampling/41.ftr


100%|██████████| 213/213 [00:02<00:00, 102.14it/s]


converting
/home/kai/data/kaggle/talkingdata/downsampling/01.ftr


# test

In [2]:
import pandas as pd
import os
import gc

PATH = '/home/kai/data/kaggle/talkingdata/haoyandata_1st/'
PATH0 = '/home/kai/data/kaggle/talkingdata/haoyandata/'
file_name = '/home/kai/data/kaggle/talkingdata/4thplace/test_cleaned1.ftr'
df = pd.read_feather(PATH0 + 'test_cleaned.ftr')
for x in os.listdir(PATH0):
    splitted = x.split('__')
    if splitted[0] == 'test_supplement':
        print(x)
        tmp = pd.read_feather(PATH0 + x)
        df[splitted[1].split('.')[0]] = tmp
        del tmp
        gc.collect()
for x in os.listdir(PATH):
    splitted = x.split('__')
    if splitted[0] == 'test_supplement':
        print(x)
        tmp = pd.read_feather(PATH + x)
        df[splitted[1].split('.')[0]] = tmp
        del tmp
        gc.collect()
print('converting')
df.to_feather(file_name)
del df
gc.collect()
print(file_name)

test_supplement__woe_ip_device_channel.ftr
test_supplement__range_count_ip_device_os_dayhourminute10.ftr
test_supplement__woe_device.ftr
test_supplement__woe_ip_app_os_channel.ftr
test_supplement__woe_app_os_channel.ftr
test_supplement__unique_ratio_day_ip_channel.ftr
test_supplement__woe_os_channel_nextClickLeakDayFlt.ftr
test_supplement__count_ip.ftr
test_supplement__count_app_device_channel_day_hour.ftr
test_supplement__variance_ip_device_hour.ftr
test_supplement__count_ratio_ip_channel.ftr
test_supplement__woe_ip_device_os_channel.ftr
test_supplement__woe_ip_device_nextClickLeakDayFlt.ftr
test_supplement__nunique_day_ip_os.ftr
test_supplement__count_app_device_day_hour.ftr
test_supplement__woe_ip_app_device_channel.ftr
test_supplement__woe_ip_app.ftr
test_supplement__woe_app_device_nextClickLeakDayFlt.ftr
test_supplement__woe_app_device_channel.ftr
test_supplement__range_count_app_os_channel_dayhourminute10.ftr
test_supplement__woe_ip_app_channel.ftr
test_supplement__range_count_ip

test_supplement__click_count_in_next_n_hour_ip_device_app_channel_1.ftr
test_supplement__click_count_in_previous_n_hour_device_app_os_6.ftr
test_supplement__click_count_in_previous_n_hour_app_channel_6.ftr
test_supplement__click_count_in_next_n_hour_device_os_1.ftr
test_supplement__click_count_in_next_n_hour_os_6.ftr
test_supplement__click_count_in_next_n_hour_device_channel_1.ftr
test_supplement__click_count_in_previous_n_hour_app_channel_1.ftr
test_supplement__click_count_in_previous_n_hour_device_6.ftr
test_supplement__click_count_in_next_n_hour_device_app_channel_6.ftr
test_supplement__click_count_in_previous_n_hour_ip_device_os_6.ftr
test_supplement__click_count_in_previous_n_hour_device_app_1.ftr
test_supplement__click_count_in_next_n_hour_ip_device_app_1.ftr
test_supplement__click_count_in_next_n_hour_ip_app_channel_1.ftr
test_supplement__click_count_in_next_n_hour_os_channel_6.ftr
test_supplement__click_count_in_previous_n_hour_ip_app_6.ftr
test_supplement__click_count_in_previ